In [1]:
import numpy as np
import pandas as pd
import os, datetime
import tensorflow as tf
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras import backend as K
print('Tensorflow version: {}'.format(tf.__version__))

import matplotlib.pyplot as plt
plt.style.use('seaborn')

import warnings
warnings.filterwarnings('ignore')
from numpy.lib.stride_tricks import as_strided

Tensorflow version: 2.4.1


# Hyperparameter

In [6]:
batch_size = 32
seq_len = 5

d_k =100
d_v = 100
n_heads = 12
ff_dim = 256

# 7个因子值处理

In [14]:
def zscore(x):
    """
    标准化
    """
    return (x - np.nanmean(x)) / np.nanstd(x)

In [15]:
def extreme_median(x):
    """
    5倍中位数去极值
    """
    md = np.nanmedian(x)
    mad = np.nanmedian(abs(x - md))
    return np.clip(x, md-5*mad, md+5*mad)

In [16]:
def fill_with_mean(x):
#     print(x)
    mean_val = np.nanmean(x)
#     x = x.fillna(mean_val,inplace = True)
#     print(x)
    x[np.isnan(x)] = mean_val
    return x

In [3]:
data = pd.read_csv('/home/hengtian/kate/signal/feature_selection_v2.csv')

In [126]:
data.columns= ['stock','date','chg','max_chg','price_40_avg','max_20_min','vol_5_avg','avg_60','max700','min12','time_f_ipo']
data = data.set_index(['stock','date'])

In [127]:
y = pd.read_csv("/home/hengtian/lucy/factor_service/basic_data/open_price_post.csv",index_col = 0)
y= y[stocks]
y = y.loc[:'2021-07-13']

In [128]:
temp =y_tmp.unstack().rename('price')
temp.index.names = ['stock','date']
df = pd.merge(data.reset_index(),temp.reset_index(),on =['stock','date'])

In [132]:
df1 = df.drop(df[(df.price==0)&(df.price.isnull())].index)
df1 = df1.drop('max700',axis = 1)

In [140]:
for stock in stocks:
    temp =df1[df1.stock ==stock]
    drop_t = temp.head(29).index
    df1 = df1.drop(drop_t)

KeyboardInterrupt: 

In [137]:
df1 = df1.drop(df1[df1.date == '2013-01-04'].index)

In [138]:
df1 = df1.drop('min12',axis = 1)

In [139]:
df1

,stock,date,chg,max_chg,price_40_avg,max_20_min,vol_5_avg,avg_60,time_f_ipo,price
1,000001.XSHE,2013-01-07,0.019387,0.022513,NaN,NaN,NaN,NaN,7950,664.263
2,000001.XSHE,2013-01-08,-0.018406,0.004295,NaN,NaN,NaN,NaN,7951,677.565
3,000001.XSHE,2013-01-09,-0.008749,0.001251,NaN,NaN,0.667132,NaN,7952,663.432
4,000001.XSHE,2013-01-10,0.000629,0.015132,NaN,NaN,0.697924,NaN,7953,659.275
5,000001.XSHE,2013-01-11,-0.020793,0.010713,NaN,NaN,0.767674,NaN,7954,659.690
...,...,...,...,...,...,...,...,...,...,...
9398587,689009.XSHG,2021-07-07,0.035966,0.038631,1.72000,1.058656,0.735744,1.425459e+08,251,74.800
9398588,689009.XSHG,2021-07-08,-0.027260,-0.001029,-0.59425,1.054854,1.065150,1.423447e+08,252,77.200
9398589,689009.XSHG,2021-07-09,-0.009914,0.012822,-1.47275,1.040326,0.612809,1.416658e+08,253,75.000
9398590,689009.XSHG,2021-07-12,0.005340,0.020561,-1.28250,1.037882,0.836393,1.424033e+08,256,75.170


In [77]:
data = data.drop(data[data.chg==0].index)

In [78]:
data = data.drop(data[data.chg==np.nan].index)

In [107]:
df1 = df1.drop(df1[df1.chg.isnull()].index)

In [111]:
len(df1[df1.price_40_avg.isnull()])/6257290 

0.02455104366267186

In [113]:
dates = df1[df1.price_40_avg.isnull()].date
for d in dates:
    avg = df1[df1.date ==d].price_40_avg.mean()
    

1          2013-01-07
2          2013-01-08
3          2013-01-09
4          2013-01-10
5          2013-01-11
              ...    
9398434    2020-11-19
9398435    2020-11-20
9398436    2020-11-23
9398437    2020-11-24
9398438    2020-11-25
Name: date, Length: 153623, dtype: object

In [114]:
df1[df1.date =='2013-01-07'].price_40_avg.mean()

nan

In [4]:
ov_inday = pd.read_csv("/home/hengtian/kate/apm/ov_1030_1430.csv",index_col = 0)
am30_inday = pd.read_csv("/home/hengtian/kate/apm/am30_1030_1430.csv",index_col = 0)

In [5]:
ideal = pd.read_csv("/home/hengtian/kate/ideal/ideal_final.csv",index_col = 0)

In [6]:
np_par = pd.read_csv("/home/hengtian/kate/growth_factor/net_profit_parent_company_q.csv",index_col = 0)

In [7]:
np_duc = pd.read_csv("/home/hengtian/kate/growth_factor/net_profit_deduct_non_recurring_pnl.csv",index_col = 0)

In [8]:
str_del_mom = pd.read_csv("/home/hengtian/kate/turnover/data/str_del_mom.csv", index_col = 0)

In [9]:
str_del_atr20 = pd.read_csv("/home/hengtian/kate/turnover/data/str_del_atr20.csv", index_col = 0)

In [10]:
ov_inday = ov_inday.loc['2018-01-31':'2021-06-22']
am30_inday =am30_inday.loc['2018-01-31':'2021-06-22']

In [11]:
ideal =ideal.loc['2018-01-31':'2021-06-22']

In [12]:
np_par=np_par.loc['2018-01-31':'2021-06-22']
np_duc=np_duc.loc['2018-01-31':'2021-06-22']
str_del_mom=str_del_mom.loc['2018-01-31':'2021-06-22']
str_del_atr20=str_del_atr20.loc['2018-01-31':'2021-06-22']

In [13]:
ov_inday_rv = ov_inday.apply(extreme_median, axis=1).apply(zscore, axis=1).apply(fill_with_mean, axis=1)
am30_inday_rv= am30_inday.apply(extreme_median, axis=1).apply(zscore, axis=1).apply(fill_with_mean, axis=1)

NameError: name 'extreme_median' is not defined

In [ ]:
idea_rv = ideal.apply(extreme_median, axis=1).apply(zscore, axis=1).apply(fill_with_mean, axis=1)

In [ ]:
np_par_rv = np_par.apply(extreme_median, axis=1).apply(zscore, axis=1).apply(fill_with_mean, axis=1)
np_duc_rv =np_duc.apply(extreme_median, axis=1).apply(zscore, axis=1).apply(fill_with_mean, axis=1)
str_m_rv =str_del_mom.apply(extreme_median, axis=1).apply(zscore, axis=1).apply(fill_with_mean, axis=1)
str_atr_rv = str_del_atr20.apply(extreme_median, axis=1).apply(zscore, axis=1).apply(fill_with_mean, axis=1)

In [ ]:
ovlp1 = ov_inday_rv.columns.intersection(idea_rv.columns)
ovlp2= ovlp1.intersection(np_par_rv.columns)
ovlp = ovlp2.intersection(str_m_rv.columns)

In [ ]:
#隔夜 + 垃圾时间
ov_inday_rv = ov_inday_rv[ovlp]
# 开盘半小时 +垃圾时间
am30_inday_rv = am30_inday_rv[ovlp]
#理想反转因子
idea_rv = idea_rv[ovlp]
#归母后净利润增长
np_par_rv = np_par_rv[ovlp]
#去损后净利润增涨
np_duc_rv =np_duc_rv[ovlp]
#换手率稳定度剔除动量
str_m_rv = str_m_rv[ovlp]
#换手率稳定度剔除ATR20
str_atr_rv =str_atr_rv[ovlp]

In [ ]:
# y值：股票收益率

# y = y.apply(extreme_median, axis=1).apply(zscore, axis=1).apply(fill_with_mean, axis=1)

In [22]:
data ={
    'apm_ov':ov_inday_rv,
    'apm_am':am30_inday_rv,
    'ideal': idea_rv,
    'np_par':np_par_rv,
    'np_duc':np_duc_rv,
    'str_m':str_m_rv,
    'str_a':str_atr_rv
}
# data.set_index(['apm_ov', 'apm_am','ideal','np_par','np_duc','str_m','str_a'],inplace=True)

In [23]:
y = y.fillna(0)

# training, test, val split

In [24]:
stocks = np.array(y.columns)
dates = np.array(y.index)
index = pd.MultiIndex.from_product([stocks, dates],
                                   names=['stock', 'dates'])

In [26]:
col =['apm_ov','apm_am','ideal','np_par','np_duc','str_m','str_a']

In [27]:
total =[]
for stock in stocks:
    factors =[]
    for comp_name, comp_df in data.items():
        factors.append(comp_df[stock].values)
    factors=np.array(factors)
#     print(factors)
    stock_factors =pd.DataFrame(factors.transpose())
#     print(stock_factors)
    total.append(stock_factors)
final = pd.concat(total,axis=0)

In [161]:
final

,0,1,2,3,4,5,6
0,0.173197,2.667298,-0.924646,-0.309227,-2.880379e-01,-0.744830,-1.209227
1,0.094186,2.817428,-1.021131,-0.309840,-2.878755e-01,-0.702784,-1.028958
2,0.359188,3.282027,-0.925552,-0.309676,-2.886072e-01,-0.735768,-0.950877
3,0.284640,3.034777,-0.881027,-0.309995,-2.888157e-01,-0.742380,-0.923904
4,-0.150260,3.120902,-1.303737,-0.310034,-2.888821e-01,-0.575085,-0.974790
...,...,...,...,...,...,...,...
816,-0.985748,-0.122660,-0.011537,0.132708,3.528894e-17,-0.462719,-1.021457
817,-0.825560,-0.185924,-0.390736,0.131015,3.528894e-17,-0.403602,-1.069677
818,-1.008121,-0.420269,0.659203,0.131662,-6.352009e-17,-0.430865,-1.110456
819,-0.760785,-0.362160,-0.211346,0.131662,-6.352009e-17,-0.370647,-1.118274


In [28]:
df = pd.DataFrame(final.values,columns = col,index= index)

In [63]:
df1

apm_ov    apm_am     ideal    np_par        np_duc  \
stock       dates                                                              
000001.XSHE 2018-01-31  0.173197  2.667298 -0.924646 -0.309227 -2.880379e-01   
            2018-02-01  0.094186  2.817428 -1.021131 -0.309840 -2.878755e-01   
            2018-02-02  0.359188  3.282027 -0.925552 -0.309676 -2.886072e-01   
            2018-02-05  0.284640  3.034777 -0.881027 -0.309995 -2.888157e-01   
            2018-02-06 -0.150260  3.120902 -1.303737 -0.310034 -2.888821e-01   
...                          ...       ...       ...       ...           ...   
689009.XSHG 2021-06-16 -0.985748 -0.122660 -0.011537  0.132708  3.528894e-17   
            2021-06-17 -0.825560 -0.185924 -0.390736  0.131015  3.528894e-17   
            2021-06-18 -1.008121 -0.420269  0.659203  0.131662 -6.352009e-17   
            2021-06-21 -0.760785 -0.362160 -0.211346  0.131662 -6.352009e-17   
            2021-06-22 -0.520619 -0.324645 -1.012530  0.131658 -6.352009e-17   

                           str_m     str_a     price  
stock       dates                                     
000001.XSHE 2018-01-31 -0.744830 -1.209227  1660.753  
            2018-02-01 -0.702784 -1.028958  1703.493  
            2018-02-02 -0.735768 -0.950877  1698.609  
            2018-02-05 -0.742380 -0.923904  1685.176  
            2018-02-06 -0.575085 -0.974790  1737.685  
...                          ...       ...       ...  
689009.XSHG 2021-06-16 -0.462719 -1.021457    79.850  
            2021-06-17 -0.403602 -1.069677    77.900  
            2021-06-18 -0.430865 -1.110456    81.480  
            2021-06-21 -0.370647 -1.118274    80.680  
            2021-06-22 -0.265041 -1.074690    79.200  

[3051699 rows x 8 columns]

In [30]:
temp =y.unstack().rename('price')
temp.index.names = ['stock','dates']
df = pd.merge(df.reset_index(),temp.reset_index(),on =['stock','dates'])

In [31]:
df = df.set_index(['stock','dates'])

In [62]:
df1 = df.drop(df[df.price==0].index)

In [59]:
df.to_csv('factor_data_v2.csv')

In [2]:
df =pd.read_csv('factor_data_v2.csv')

In [3]:
df = df.drop(['np_par','np_duc'],axis =1)

In [4]:
y = pd.read_csv('/home/hengtian/kate/transformer/data_v1/y_value.csv',index_col = 0)

In [27]:
df_train[df_train.stock =='600419.XSHG']

,stock,dates,apm_ov,apm_am,ideal,str_m,str_a,price
2057172,600419.XSHG,2018-01-31,1.812609e-17,4.696251e-18,-0.295962,-0.814484,-0.978260,56.929
2057173,600419.XSHG,2018-02-01,2.264317e-17,-1.874783e-17,-0.003865,-0.888168,-0.866965,56.929
2057174,600419.XSHG,2018-02-02,4.499954e-18,2.334240e-17,0.009501,-0.889514,-0.898942,53.234
2057175,600419.XSHG,2018-02-05,2.725867e-17,4.699357e-18,-0.471347,-0.870886,-0.793529,52.276
2057176,600419.XSHG,2018-02-06,-9.312487e-18,-1.443215e-17,-0.621703,-0.965863,-0.823936,49.101
...,...,...,...,...,...,...,...,...
2057817,600419.XSHG,2020-10-14,-1.589554e-01,5.125658e-01,-0.782102,0.090663,0.788353,50.178
2057818,600419.XSHG,2020-10-15,7.884119e-02,1.113130e+00,-0.970001,-0.236644,0.392542,51.170
2057819,600419.XSHG,2020-10-16,2.929215e-01,1.000842e+00,-1.644472,-0.455822,0.115654,52.162
2057820,600419.XSHG,2020-10-19,1.793008e-01,7.684647e-01,-1.845314,-0.427269,0.120876,52.162


In [61]:
y.to_csv('y_value.csv')

In [5]:
times = sorted(y.index.values)
last_10pct = sorted(y.index.values)[-int(0.1*len(times))] # Last 10% of series
last_20pct = sorted(y.index.values)[-int(0.2*len(times))] # Last 20% of series

In [6]:
df_train = df[(df.dates< last_20pct)]  # Training data are 80% of total data
df_val = df[(df.dates >= last_20pct) & (df.dates < last_10pct)]
df_test = df[(df.dates >= last_10pct)]

In [9]:
np_train = np.array(df_train[['apm_ov','ideal','apm_am','str_m','str_a']])

In [13]:
np_train[:20,].itemsize

8

In [ ]:

# random.seed(5)
# prices = np.random.randn(100)
# We want closing prices from the ten days prior
window = 20
# Create array of closing prices to predict
# y = prices[window:]

def make_X2():
    # Save stride (num bytes) between each item
    stride1,stride2 = np_train.strides
    desired_shape = [len(np_train) - window+1, window,5]
    # Get a view of the prices with shape desired_shape, strides as defined, don't write to original array 
    X2 = as_strided(np_train, desired_shape, strides=[stride2,stride1,stride2], writeable=False)
    return X2 

make_X2()
# print(make_X2().shape)
# timeit(make_X1) # 56.7 seconds 
# timeit(make_X2) # 7.7 seconds, over 7x faster!

In [11]:
np_train.strides

(8, 18940840)

In [8]:
#每一只股票，每一天，都有一张5X7的表格
X_train,y_train =[],[]
for stock in set(df_train['stock']):
    temp = df_train[df_train['stock']==stock]
    for i in range(25,len(temp)):
        ct = temp[i-25:i-5].drop(['stock','dates','price'],axis =1)
        X_train.append(ct)
        rtn = temp['price'].iloc[i]/temp['price'].iloc[i-5]-1
        y_train.append(rtn)

In [9]:
X_val,y_val =[],[]
for stock in set(df_val['stock']):
    temp = df_val[df_val['stock']==stock]
    for i in range(25,len(temp)):
        ct = temp[i-25:i-5].drop(['stock','dates','price'],axis =1)
        X_val.append(ct)
        rtn = temp['price'].iloc[i]/temp['price'].iloc[i-5]-1
        y_val.append(rtn)

In [10]:
X_test,y_test =[],[]
for stock in set(df_test['stock']):
    temp = df_test[df_test['stock']==stock]
    for i in range(25,len(temp)):
        ct = temp[i-25:i-5].drop(['stock','dates','price'],axis =1)
        X_test.append(ct)
        rtn = temp['price'].iloc[i]/temp['price'].iloc[i-5]-1
        y_test.append(rtn)

In [14]:
X_train, y_train = np.array(X_train), np.array(y_train)
X_val, y_val = np.array(X_val), np.array(y_val)
X_test, y_test = np.array(X_test), np.array(y_test)

In [17]:
print('Training set shape', X_train.shape, y_train.shape)
print('Validation set shape', X_val.shape, y_val.shape)
print('Testing set shape' ,X_test.shape, y_test.shape)

Training set shape (2266268, 20, 5) (2266268,)
Validation set shape (231923, 20, 5) (231923,)
Testing set shape (240192, 20, 5) (240192,)


In [18]:
np.save('X_train',X_train)
np.save('y_train',y_train)
np.save('X_val',X_val)
np.save('y_val',y_val)
np.save('X_test',X_test)
np.save('y_test',y_test)

In [65]:
X_train = np.load('X_train.npy')
y_train = np.load('y_train.npy')
X_val = np.load('X_val.npy')
y_val = np.load('y_val.npy')
X_test = np.load('X_test.npy')
y_test = np.load('y_test.npy')

In [22]:
tp_train[tp_train['stock']=='600419.XSHG'][0:9]

,index,stock,dates,apm_ov,apm_am,ideal,str_m,str_a,price
1614524,2057172,600419.XSHG,2018-01-31,1.812609e-17,4.696251e-18,-0.295962,-0.814484,-0.978260,56.929
1614525,2057173,600419.XSHG,2018-02-01,2.264317e-17,-1.874783e-17,-0.003865,-0.888168,-0.866965,56.929
1614526,2057174,600419.XSHG,2018-02-02,4.499954e-18,2.334240e-17,0.009501,-0.889514,-0.898942,53.234
1614527,2057175,600419.XSHG,2018-02-05,2.725867e-17,4.699357e-18,-0.471347,-0.870886,-0.793529,52.276
1614528,2057176,600419.XSHG,2018-02-06,-9.312487e-18,-1.443215e-17,-0.621703,-0.965863,-0.823936,49.101
1614529,2057177,600419.XSHG,2018-02-07,4.671550e-18,-1.455036e-17,-0.674755,-0.979141,-0.847188,50.566
1614530,2057178,600419.XSHG,2018-02-08,2.102197e-17,-2.415169e-17,-0.716145,-0.982927,-0.860653,49.416
1614531,2057179,600419.XSHG,2018-02-09,-2.872814e-17,4.927481e-18,-0.997665,-0.981082,-0.915727,49.813
1614532,2057180,600419.XSHG,2018-02-12,-3.802235e-17,1.466044e-17,-0.767367,-0.975585,-0.829697,49.266


# Time2Vector

In [42]:
class Time2Vector(Layer):
  def __init__(self, seq_len, **kwargs):
    super(Time2Vector, self).__init__()
    self.seq_len = seq_len

  def build(self, input_shape):
    '''Initialize weights and biases with shape (batch, seq_len)'''
    self.weights_linear = self.add_weight(name='weight_linear',
                                shape=(int(self.seq_len),),
                                initializer='uniform',
                                trainable=True)
    
    self.bias_linear = self.add_weight(name='bias_linear',
                                shape=(int(self.seq_len),),
                                initializer='uniform',
                                trainable=True)
    
    self.weights_periodic = self.add_weight(name='weight_periodic',
                                shape=(int(self.seq_len),),
                                initializer='uniform',
                                trainable=True)

    self.bias_periodic = self.add_weight(name='bias_periodic',
                                shape=(int(self.seq_len),),
                                initializer='uniform',
                                trainable=True)

  def call(self, x):
    '''Calculate linear and periodic time features'''
    x = tf.math.reduce_mean(x[:,:,:4], axis=-1) 
    time_linear = self.weights_linear * x + self.bias_linear # Linear time feature
    time_linear = tf.expand_dims(time_linear, axis=-1) # Add dimension (batch, seq_len, 1)
    
    time_periodic = tf.math.sin(tf.multiply(x, self.weights_periodic) + self.bias_periodic)
    time_periodic = tf.expand_dims(time_periodic, axis=-1) # Add dimension (batch, seq_len, 1)
    return tf.concat([time_linear, time_periodic], axis=-1) # shape = (batch, seq_len, 2)
   
  def get_config(self): # Needed for saving and loading model with custom layer
    config = super().get_config().copy()
    config.update({'seq_len': self.seq_len})
    return config

# SingleAttention

In [43]:
class SingleAttention(Layer):
  def __init__(self, d_k, d_v):
    super(SingleAttention, self).__init__()
    self.d_k = d_k
    self.d_v = d_v

  def build(self, input_shape):
    self.query = Dense(self.d_k, 
                       input_shape=input_shape, 
                       kernel_initializer='glorot_uniform', 
                       bias_initializer='glorot_uniform')
    
    self.key = Dense(self.d_k, 
                     input_shape=input_shape, 
                     kernel_initializer='glorot_uniform', 
                     bias_initializer='glorot_uniform')
    
    self.value = Dense(self.d_v, 
                       input_shape=input_shape, 
                       kernel_initializer='glorot_uniform', 
                       bias_initializer='glorot_uniform')

  def call(self, inputs): # inputs = (in_seq, in_seq, in_seq)
    q = self.query(inputs[0])
    k = self.key(inputs[1])

    attn_weights = tf.matmul(q, k, transpose_b=True)
    attn_weights = tf.map_fn(lambda x: x/np.sqrt(self.d_k), attn_weights)
    attn_weights = tf.nn.softmax(attn_weights, axis=-1)
    
    v = self.value(inputs[2])
    attn_out = tf.matmul(attn_weights, v)
    return attn_out    

# MultiHead

In [44]:
class MultiAttention(Layer):
  def __init__(self, d_k, d_v, n_heads):
    super(MultiAttention, self).__init__()
    self.d_k = d_k
    self.d_v = d_v
    self.n_heads = n_heads
    self.attn_heads = list()

  def build(self, input_shape):
    for n in range(self.n_heads):
      self.attn_heads.append(SingleAttention(self.d_k, self.d_v))  
    
    # input_shape[0]=(batch, seq_len, 7), input_shape[0][-1]=7 
    self.linear = Dense(input_shape[0][-1], 
                        input_shape=input_shape, 
                        kernel_initializer='glorot_uniform', 
                        bias_initializer='glorot_uniform')

  def call(self, inputs):
    attn = [self.attn_heads[i](inputs) for i in range(self.n_heads)]
    concat_attn = tf.concat(attn, axis=-1)
    multi_linear = self.linear(concat_attn)
    return multi_linear 

# Encoder

In [45]:
class TransformerEncoder(Layer):
  def __init__(self, d_k, d_v, n_heads, ff_dim, dropout=0.1, **kwargs):
    super(TransformerEncoder, self).__init__()
    self.d_k = d_k
    self.d_v = d_v
    self.n_heads = n_heads
    self.ff_dim = ff_dim
    self.attn_heads = list()
    self.dropout_rate = dropout

  def build(self, input_shape):
    self.attn_multi = MultiAttention(self.d_k, self.d_v, self.n_heads)
    self.attn_dropout = Dropout(self.dropout_rate)
    self.attn_normalize = LayerNormalization(input_shape=input_shape, epsilon=1e-6)

    self.ff_conv1D_1 = Conv1D(filters=self.ff_dim, kernel_size=1, activation='relu')
    # input_shape[0]=(batch, seq_len, 7), input_shape[0][-1] = 7 
    self.ff_conv1D_2 = Conv1D(filters=input_shape[0][-1], kernel_size=1) 
    self.ff_dropout = Dropout(self.dropout_rate)
    self.ff_normalize = LayerNormalization(input_shape=input_shape, epsilon=1e-6)    
  
  def call(self, inputs): # inputs = (in_seq, in_seq, in_seq)
    attn_layer = self.attn_multi(inputs)
    attn_layer = self.attn_dropout(attn_layer)
    attn_layer = self.attn_normalize(inputs[0] + attn_layer)

    ff_layer = self.ff_conv1D_1(attn_layer)
    ff_layer = self.ff_conv1D_2(ff_layer)
    ff_layer = self.ff_dropout(ff_layer)
    ff_layer = self.ff_normalize(inputs[0] + ff_layer)
    return ff_layer 

  def get_config(self): # Needed for saving and loading model with custom layer
    config = super().get_config().copy()
    config.update({'d_k': self.d_k,
                   'd_v': self.d_v,
                   'n_heads': self.n_heads,
                   'ff_dim': self.ff_dim,
                   'attn_heads': self.attn_heads,
                   'dropout_rate': self.dropout_rate})
    return config

# Dataset

In [141]:
X_train = np.load('X_train.npy')
y_train = np.load('y_train.npy')
X_val = np.load('X_val.npy')
y_val = np.load('y_val.npy')
X_test = np.load('X_test.npy')
y_test = np.load('y_test.npy')

In [142]:
y_train = y_train.reshape((2334994,1))
y_val = y_val.reshape((302449,1))
y_test = y_test.reshape((313588,1))

In [143]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val))
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))

In [144]:
X_val_test = X_val[:10000]
y_val_test = y_val[:10000]

In [145]:
test_val =  tf.data.Dataset.from_tensor_slices((X_val_test, y_val_test))

# Model

In [58]:
def create_model():
  '''Initialize time and transformer layers'''
  time_embedding = Time2Vector(seq_len)
  attn_layer1 = TransformerEncoder(d_k, d_v, n_heads, ff_dim)
  attn_layer2 = TransformerEncoder(d_k, d_v, n_heads, ff_dim)
  attn_layer3 = TransformerEncoder(d_k, d_v, n_heads, ff_dim)


  '''Construct model'''
  in_seq = Input(shape=(seq_len, 7))
  x = time_embedding(in_seq)
  x = Concatenate(axis=-1)([in_seq, x])
  x = attn_layer1((x, x, x))
  x = attn_layer2((x, x, x))
  x = attn_layer3((x, x, x))
  x = GlobalAveragePooling1D(data_format='channels_first')(x)
  x = Dropout(0.1)(x)
  x = Dense(64, activation='relu')(x)
  x = Dropout(0.1)(x)
  out = Dense(1, activation='linear')(x)

  model = Model(inputs=in_seq, outputs=out)
  model.compile(loss='mse', optimizer='adam', metrics=['mae', 'mape'])
  return model


model = create_model()
model.summary()

callback = tf.keras.callbacks.ModelCheckpoint('Transformer+TimeEmbedding.hdf5', 
                                              monitor='val_loss', 
                                              save_best_only=True, verbose=1)

history = model.fit(X_train, y_train, 
                    batch_size=batch_size, 
                    epochs=5, 
                    callbacks=[callback],
                    validation_data=(X_val, y_val))  

tf.config.threading.set_intra_op_parallelism_threads(2)
tf.config.threading.set_inter_op_parallelism_threads(2)

# with tf.device('/CPU:4'):
# model = tf.keras.models.load_model('/content/Transformer+TimeEmbedding.hdf5',
#                                    custom_objects={'Time2Vector': Time2Vector, 
#                                                    'SingleAttention': SingleAttention,
#                                                    'MultiAttention': MultiAttention,
#                                                    'TransformerEncoder': TransformerEncoder})
#####################################################################################################
train_loss_results = []
train_accuracy_results = []

num_epochs = 5

for epoch in range(num_epochs):
    epoch_loss_avg = tf.keras.metrics.Mean()
    epoch_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
    # Training loop - using batches of 32
    for x, y in train_dataset:
        # 优化模型
        loss_value, grads = grad(model, x, y)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

    # 追踪进度
        epoch_loss_avg(loss_value)  # 添加当前的 batch loss
    # 比较预测标签与真实标签
        epoch_accuracy(y, model(x))
    for (x, y) in test_val:
        logits = model(x)
        prediction = tf.argmax(logits, axis=1, output_type=tf.int32)
        test_accuracy(prediction, y)
    print("Test set accuracy: {:.3%}".format(test_accuracy.result()))
  # 循环结束
    train_loss_results.append(epoch_loss_avg.result())
    train_accuracy_results.append(epoch_accuracy.result())
    if epoch % 50 == 0:
        print("Epoch {:03d}: Loss: {:.3f}, Accuracy: {:.3%}".format(epoch,
                                                                epoch_loss_avg.result(),
                                                                epoch_accuracy.result()))


###############################################################################
'''Calculate predictions and metrics'''

#Calculate predication for training, validation and test data
train_pred = model.predict(X_train)
val_pred = model.predict(X_val)
test_pred = model.predict(X_test)

#Print evaluation metrics for all datasets
train_eval = model.evaluate(X_train, y_train, verbose=0)
val_eval = model.evaluate(X_val, y_val, verbose=0)
test_eval = model.evaluate(X_test, y_test, verbose=0)
print(' ')
print('Evaluation metrics')
print('Training Data - Loss: {:.4f}, MAE: {:.4f}, MAPE: {:.4f}'.format(train_eval[0], train_eval[1], train_eval[2]))
print('Validation Data - Loss: {:.4f}, MAE: {:.4f}, MAPE: {:.4f}'.format(val_eval[0], val_eval[1], val_eval[2]))
print('Test Data - Loss: {:.4f}, MAE: {:.4f}, MAPE: {:.4f}'.format(test_eval[0], test_eval[1], test_eval[2]))

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 5, 7)]       0                                            
__________________________________________________________________________________________________
time2_vector_3 (Time2Vector)    (None, 5, 2)         20          input_4[0][0]                    
__________________________________________________________________________________________________
concatenate_3 (Concatenate)     (None, 5, 9)         0           input_4[0][0]                    
                                                                 time2_vector_3[0][0]             
__________________________________________________________________________________________________
transformer_encoder_3 (Transfor (None, 5, 9)         51718       concatenate_3[0][0]        

KeyboardInterrupt: 

In [ ]:
def grad(model, inputs, targets):
    with tf.GradientTape() as tape:
    loss_value = loss(model, inputs, targets)
    return loss_value, tape.gradient(loss_value, model.trainable_variables)

# Fit 函数

In [ ]:
def fit(x, y, batch_size, epochs, verbose=1, callbacks, validation_split, validation_data, shuffle=True, class_weight=None, 
sample_weight=None, initial_epoch=0, steps_per_epoch=None, validation_steps=None, validation_freq=1):
    batch_size = self._validate_or_infer_batch_size(batch_size, steps_per_epoch, x)
    x, y, sample_weights = self._standardize_user_data(
            x, y,
            sample_weight=sample_weight,
            class_weight=class_weight,
            batch_size=batch_size)
    do_validation = False
    if validation_data:
        do_validation = True
        if len(validation_data) == 2:
            val_x, val_y = validation_data
            val_sample_weight = None
        elif len(validation_data) == 3:
            val_x, val_y, val_sample_weight = validation_data
        else:
            raise ValueError('When passing validation_data, '
                             'it must contain 2 (x_val, y_val) '
                             'or 3 (x_val, y_val, val_sample_weights) '
                             'items, however it contains %d items' %
                             len(validation_data))

        val_x, val_y, val_sample_weights = self._standardize_user_data(
            val_x, val_y,
            sample_weight=val_sample_weight,
            batch_size=batch_size)
        if self._uses_dynamic_learning_phase():
            val_inputs = val_x + val_y + val_sample_weights + [0.]
        else:
            val_inputs = val_x + val_y + val_sample_weights
    if self._uses_dynamic_learning_phase():
        fit_inputs = x + y + sample_weights + [1.]
    else:
        fit_inputs = x + y + sample_weights
    self._make_train_function()
    fit_function = self.train_function

In [ ]:
def _validate_or_infer_batch_size(self, batch_size, steps, x):
 
    if batch_size is not None and is_generator_or_sequence(x):
        raise ValueError('The `batch_size` argument must not be specified when'
                         ' using a generator or Sequence as an input.')

    layers = super(Model, self).layers  # Avoids the override in Sequential.
    if layers:
        first_layer = layers[0]
        static_batch_size = get_static_batch_size(first_layer)
        if static_batch_size is not None:

            # Check `batch_size` argument is consistent with InputLayer.
            if batch_size is not None and batch_size != static_batch_size:
                raise ValueError('The `batch_size` argument value {} is '
                                 'incompatible with the specified batch '
                                 'size of your Input Layer: {}'
                                 .format(batch_size, static_batch_size))

            # Set inferred batch size from the InputLayer.
            if steps is None:
                batch_size = static_batch_size

    if batch_size is None and steps is None:
        # Backwards compatibility
        batch_size = 32
    return batch_size

In [ ]:
def _make_train_function(self):
    if not hasattr(self, 'train_function'):
        raise RuntimeError('You must compile your model before using it.')
    self._check_trainable_weights_consistency()
    if self.train_function is None:
        inputs = (self._feed_inputs +
                  self._feed_targets +
                  self._feed_sample_weights)
        if self._uses_dynamic_learning_phase():
            inputs += [K.learning_phase()]

        with K.name_scope('training'):
            with K.name_scope(self.optimizer.__class__.__name__):
                training_updates = self.optimizer.get_updates(
                    params=self._collected_trainable_weights,
                    loss=self.total_loss)
            updates = (self.updates +
                       training_updates +
                       self.metrics_updates)
            # Gets loss and metrics. Updates weights at each call.
            self.train_function = K.function(
                inputs,
                [self.total_loss] + self.metrics_tensors,
                updates=updates,
                name='train_function',
                **self._function_kwargs)

In [ ]:
def function(inputs, outputs, updates=None, **kwargs):
    if kwargs:
        for key in kwargs:
            session_has_key = has_arg(tf.Session.run, key, True)
            function_has_key = has_arg(Function.__init__, key, True)
            if not (session_has_key or function_has_key):
                raise ValueError('Invalid argument "%s" passed to K.function '
                                 'with TensorFlow backend' % key)
    return Function(inputs, outputs, updates=updates, **kwargs)